# Lecture 6:  Wednesday, September 20th 2017
## Towards Intermediate Python
Topics:
* Nested environments
* Closures
* Decorators
* Recap:  How does this stuff really work?

Before we begin, there is something that you should know about.  It's [http://pythontutor.com/](http://pythontutor.com/) and it's a great way to learn what is going on under the hood when you write Python.  You can visualize the Python code you write:  [Visualize](http://pythontutor.com/visualize.html#mode=edit).  I'll have you test out and visualize some very small scripts using that website.

What follows is an example of how to do this.  The example below is attempting to show how variables (actually called _names_ in Python) are stored in Python.  Notice that the list `b` was assigned to the list `a` and that `a` and `b` are identical (they have the same ids).  Because of this, when we append a number to `a`, `b` is also changed!

Note:  When trying to embed HTML into your notebook, you need to use the syntax:  `HTML('url')`.  `pythontutor` has a `Generate embed code` button which will generate the necessary code to embed into your webpage.

In [17]:
from IPython.display import HTML # Allows us to embed HTML into our notebook.

In [18]:
HTML('<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=a%20%3D%20%5B1,%203,%205%5D%0Ab%20%3D%20a%0Aprint%28%22a%20%3D%20%7B0%7D%20and%20has%20id%20%7B1%7D%22.format%28a,%20id%28a%29%29%29%0Aprint%28%22b%20%3D%20%7B0%7D%20and%20has%20id%20%7B1%7D%22.format%28b,%20id%28b%29%29%29%0Aprint%28%22Is%20b%20a%3F%20%7B0%7D%22.format%28b%20is%20a%29%29%0A%0Aa.append%287%29%0Aprint%28%22a%20%3D%20%7B%7D%22.format%28a%29%29%0Aprint%28%22b%20%3D%20%7B%7D%22.format%28b%29%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=0&heapPrimitives=false&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>')

So what is going on?  Well, Python variables are reference variables.  You could say "the variable `a` (`b`) is assigned to a list" rather than "the list is assigned to the variable `a` (`b`)".

From the Python Language Reference, section 3.1:

>Every object has an identity, a type and a value. An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The ‘is‘ operator compares the identity of two objects; the id() function returns an integer representing its identity (currently implemented as its address).

## Nested Environments
You can nest the definitions of functions. When you do this, inner function definitions are not even evaluated until the outer function is called. These inner functions have access to the name bindings in the scope of the outer function. So below, in `make_statements`, both `s` and `key` will be defined. And in `key`, you have access to `s`. This sharing is called *lexical scoping*. 

In [3]:
def make_statement(s):
    def key(k):
        c=(s, k)
        return c
    return key
k = make_statement('name: ')
#we have captured the first element of the tuple as a "kind of state"
name = k('Albert')
print(name)
name2 = k('Emmy')
print(name2)

('name: ', 'Albert')
('name: ', 'Emmy')


We can make this a little bit more explicit.  In the line `k = make_statement('name: ')`, `make_statement()` has returned the inner function `key` and the inner function has been given the name `k`.  Now, when we call `k()` the inner function returns the desired tuple.

The reason this works is that in addition to the environment in which a user-defined function is running, that function has access to a second environment: the environment in which the function was defined.  Here, `key` has access to the environment of `make_statement`. In this sense the environment of `make_statement` is the parent of the environment of `key`.

This enables two things:

1. Names inside the inner functions (or the outer ones for that matter) do not interfere with names in the global scope. Inside the outer and inner functions, the "most lexically local" names are the ones that matter
2. An inner function can access the environment of its enclosing (outer) function

Earlier, the phrase `name binding` was used.  See [Name Binding](https://en.wikipedia.org/wiki/Name_binding) for a high level overview of the concept.  Name binding is the association between a name and an object (or value): [Python Variables](https://mathieularose.com/python-variables/).  Here's a little more on that topic from the [Python execution model](https://docs.python.org/3/reference/executionmodel.html):

#### The binding of names

>The following constructs bind names: formal parameters to functions, import statements, class and function definitions (these bind the class or function name in the defining block), and targets that are identifiers if occurring in an assignment, for loop header, or after as in a with statement or except clause. The import statement of the form from ... import * binds all names defined in the imported module, except those beginning with an underscore. This form may only be used at the module level.

>If a name is bound in a block, it is a local variable of that block, unless declared as nonlocal or global. If a name is bound at the module level, it is a global variable. (The variables of the module code block are local and global.) If a variable is used in a code block but not defined there, it is a free variable.

The idea of `scope` was also mentioned.  Here is a bit more information on `scope`:

#### The lookup of names

> A scope defines the visibility of a name within a block. If a local variable is defined in a block, its scope includes that block. If the definition occurs in a function block, the scope extends to any blocks contained within the defining one, unless a contained block introduces a different binding for the name.

>When a name is used in a code block, it is resolved using the nearest enclosing scope. The set of all such scopes visible to a code block is called the block’s environment.